In [3]:
import os
import sys
from Flight_Fare.exception import CustomException
from Flight_Fare.logger import logging
from Flight_Fare.config.configuration import Configuration
from Flight_Fare.entity.artifact_entity import *
from Flight_Fare.entity.config_entity import *
from Flight_Fare.constant import *
from Flight_Fare.util.util import *
from sklearn.impute import SimpleImputer

from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler,OneHotEncoder,LabelEncoder
from sklearn.pipeline import Pipeline
import pandas as pd 

'1.1.4'

In [25]:
train_df = pd.read_csv(r"Flight_Fare\artifact\data_ingestion\2023-07-25-01-27-30\ingested_data\train\Flight_Fare_Prediction.csv")
test_df = pd.read_csv(r"Flight_Fare\artifact\data_ingestion\2023-07-25-01-27-30\ingested_data\test\Flight_Fare_Prediction.csv")

In [52]:
X = train_df.drop(['Price','Arrival_Time','Duration'],axis = 1)

In [27]:
X['journey_day'] = X['Date_of_Journey'].str.split('/').str[0].astype(int)
X['journey_month'] = X['Date_of_Journey'].str.split('/').str[1].str[1].astype(int)

X.drop('Date_of_Journey',inplace = True,axis = 1)

In [28]:
X['journey_day']

,Airline,Source,Destination,Route,Total_Stops,Additional_Info,journey_day,journey_month
0,Jet Airways,Mumbai,Hyderabad,BOM → HYD,non-stop,No info,27,6
1,Multiple carriers,Delhi,Cochin,DEL → BOM → COK,1 stop,No info,3,3
2,Multiple carriers,Delhi,Cochin,DEL → BOM → COK,1 stop,No info,27,5
3,Air Asia,Kolkata,Banglore,CCU → DEL → BLR,1 stop,No info,9,5
4,Air India,Delhi,Cochin,DEL → AMD → BOM → COK,2 stops,No info,9,6
...,...,...,...,...,...,...,...,...
7473,Jet Airways,Delhi,Cochin,DEL → BOM → COK,1 stop,No info,27,3
7474,Jet Airways,Kolkata,Banglore,CCU → BOM → BLR,1 stop,In-flight meal not included,9,5
7475,Multiple carriers,Delhi,Cochin,DEL → BOM → COK,1 stop,No info,15,5
7476,IndiGo,Banglore,New Delhi,BLR → DEL,non-stop,No info,3,3


In [53]:
class FeatureGenerator():

    def __init__(self,columns):
        self.columns = columns

    def fit(self,X,y=None):
        return X

    def transform(self):
        return self.columns
        

In [5]:
col = ['Airline','Source','Destination','Total_Stops']
cat_pipeline = Pipeline(steps=[
                 ('one_hot_encoder', OneHotEncoder()),
                 ('scaler', StandardScaler(with_mean=False))
            ]
            )
preprocessing = ColumnTransformer(transformers = [
                ('cat_pipeline', cat_pipeline, col)
            ])
preprocessing.fit(X)

ColumnTransformer(transformers=[('cat_pipeline',
                                 Pipeline(steps=[('one_hot_encoder',
                                                  OneHotEncoder()),
                                                 ('scaler',
                                                  StandardScaler(with_mean=False))]),
                                 ['Airline', 'Source', 'Destination',
                                  'Total_Stops'])])

In [6]:
preprocessing.fit_transform(X)

<7478x29 sparse matrix of type '<class 'numpy.float64'>'
	with 29912 stored elements in Compressed Sparse Row format>

In [54]:
cat_one = ['Airline','Source','Destination','Total_Stops']
num_cols = ['Dep_Time']


num_pipeline = Pipeline(steps=[
                ('feature_generator', FeatureGenerator(num_cols)),
                ('scaler', StandardScaler())
            ]
            )

cat_pipeline = Pipeline(steps=[
        ('one_hot_encoder', OneHotEncoder()),
        ('scaler', StandardScaler(with_mean=False))
]
)



preprocessing = ColumnTransformer(transformers = [
                ('num_pipeline', num_pipeline, num_cols),
                ('cat_pipeline', cat_pipeline, cat_one)
               
            ])



In [55]:
preprocessing.fit_transform(X)

AttributeError: 'Dep_Time' is not a valid function for 'Series' object

In [44]:
X['Day'] = 0
for i in range(len(X['Date_of_Journey'])):
    X['Day'][i] = X['Date_of_Journey'][i] .split("/")[0]
X
    

C:\Users\HP-LAPTOP\AppData\Local\Temp\ipykernel_4180\36958048.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Day'][i] = X['Date_of_Journey'][i] .split("/")[0]


,Airline,Date_of_Journey,Source,Destination,Route,Total_Stops,Additional_Info,Day
0,Jet Airways,27/06/2019,Mumbai,Hyderabad,BOM → HYD,non-stop,No info,27
1,Multiple carriers,3/03/2019,Delhi,Cochin,DEL → BOM → COK,1 stop,No info,3
2,Multiple carriers,27/05/2019,Delhi,Cochin,DEL → BOM → COK,1 stop,No info,27
3,Air Asia,9/05/2019,Kolkata,Banglore,CCU → DEL → BLR,1 stop,No info,9
4,Air India,9/06/2019,Delhi,Cochin,DEL → AMD → BOM → COK,2 stops,No info,9
...,...,...,...,...,...,...,...,...
7473,Jet Airways,27/03/2019,Delhi,Cochin,DEL → BOM → COK,1 stop,No info,27
7474,Jet Airways,9/05/2019,Kolkata,Banglore,CCU → BOM → BLR,1 stop,In-flight meal not included,9
7475,Multiple carriers,15/05/2019,Delhi,Cochin,DEL → BOM → COK,1 stop,No info,15
7476,IndiGo,03/03/2019,Banglore,New Delhi,BLR → DEL,non-stop,No info,03
